(raga-recognition)=
# Automatic raga recognition

Intro here...

In [ ]:
%%capture
%pip install compiam

import compiam

In [ ]:
from compiam.melody import DEEPSRGM
deepsrgm = DEEPSRGM()  # initializing a melodia instance
feat = deepsrgm.get_features(from_mirdata=True, track_id="TODO")
deepsrgm.predict(feat)
